<a href="https://colab.research.google.com/github/NNOSS999/Gemma3-4B/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

要运行此内容，请在 **免费的** Tesla T4 Google Colab 实例中点击 "*Runtime*"，然后点击 "*Run all*"！
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> 如需帮助请加入 Discord + ⭐ *在 <a href="https://github.com/unslothai/unsloth">Github</a> </i> 上为我们点星* ⭐
</div>

要在自己的电脑上安装 Unsloth，请按照我们 Github 页面上的[安装说明](https://docs.unsloth.ai/get-started/installing-+-updating)操作。

您将学习如何[准备数据](#Data)、如何[训练](#Train)、如何[运行模型](#Inference)以及[如何保存模型](#Save)。

### 新闻

阅读我们的 [Gemma 3 博客](https://unsloth.ai/blog/gemma3)，了解 Unsloth 的最新动态，
以及我们的 [推理博客](https://unsloth.ai/blog/r1-reasoning)，学习如何训练推理模型。

访问我们的文档，查看所有 [模型上传](https://docs.unsloth.ai/get-started/all-our-models)
和 [Notebook 教程](https://docs.unsloth.ai/get-started/unsloth-notebooks)。


#### 安装

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

`FastModel` 现在几乎支持加载任何模型！这包括视觉模型和文本模型！

In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-21 09:52:52 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

我们现在新增了LoRA适配器，只需更新少量参数即可完成微调！

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### 数据准备
我们现在使用 `Gemma-3` 格式进行对话式微调。我们采用 [Maxime Labonne 的 FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) 数据集（ShareGPT 风格）。Gemma-3 的多轮对话格式如下：

```
<bos><start_of_turn>user
你好！<end_of_turn>
<start_of_turn>model
嗨！<end_of_turn>
```


我们使用 `get_chat_template` 函数获取正确的对话模板。支持的模板包括：`zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` 等。

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [30]:
from datasets import load_dataset
dataset = load_dataset("BlcaCola/YI-Dataset-Chinese-Basics", split = "train")

In [31]:
print(dataset[0])

{'conversations': [{'user': '你好，最近怎么样？', 'assistant': '你好！我最近还不错，谢谢。', 'history': None, 'response': None}]}


我们现在使用 `standardize_data_formats` 功能来自动将数据集转换为适合微调的标准格式！

In [32]:
def convert_for_unsloth(example):
    output = []
    if isinstance(example["conversations"], list):
        for turn in example["conversations"]:
            if "user" in turn:
                output.append({"role": "user", "content": turn["user"]})
            if "assistant" in turn:
                output.append({"role": "assistant", "content": turn["assistant"]})
    return {"conversations": output}

# ⚠️ 注意不要加 batched=True
dataset = dataset.map(convert_for_unsloth)





Map:   0%|          | 0/10185 [00:00<?, ? examples/s]

In [33]:
print(dataset[0])


{'conversations': [{'content': '你好，最近怎么样？', 'role': 'user'}, {'content': '你好！我最近还不错，谢谢。', 'role': 'assistant'}]}


In [34]:
from unsloth import standardize_data_formats

dataset = standardize_data_formats(dataset)


Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/10185 [00:00<?, ? examples/s]

来看看第100行的数据是什么样的！

In [35]:
dataset[100]

{'conversations': [{'content': '昆明是中国的哪个省的省会城市？', 'role': 'user'},
  {'content': '昆明是云南省的省会城市。', 'role': 'assistant'}]}

现在我们需要为对话应用`Gemma-3`的聊天模板，并将其保存到`text`字段中。

In [44]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/10185 [00:00<?, ? examples/s]

来看看聊天模板的效果吧！注意 `Gemma-3` 默认会添加 `<bos>` 标记！

In [45]:
dataset[100]["text"]

'<bos><start_of_turn>user\n昆明是中国的哪个省的省会城市？<end_of_turn>\n<start_of_turn>model\n昆明是云南省的省会城市。<end_of_turn>\n'

In [46]:
# 示例: 计算整个训练集的token总数
total_token_count = 0

# 遍历每条数据
for example in dataset:
    # 取出文本字段，并计算 token 数量
    text = example["text"]  # 或者根据你的数据集修改这个字段名
    encoded = tokenizer(text)
    total_token_count += len(encoded["input_ids"])

print("Total tokens in the dataset:", total_token_count)


Total tokens in the dataset: 10185


<a name="Train"></a>
### 模型训练

现在让我们使用 Huggingface TRL 的 `SFTTrainer`！更多文档详见：[TRL SFT 文档](https://huggingface.co/docs/trl/sft_trainer)。我们这里只进行 60 步训练以加快速度，但您可以设置 `num_train_epochs=1` 进行完整训练，并将 `max_steps=None` 关闭。

In [47]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 6, # Use GA to mimic batch size!
        warmup_steps = 250,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 3000,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/10185 [00:00<?, ? examples/s]

我们还使用了 Unsloth 的 `train_on_completions` 方法，仅针对助手输出进行训练，忽略用户输入的损失计算。这有助于提高微调的准确性！

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

让我们确认指令部分的掩码处理已完成！再次打印第100行数据：

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>\n<start_of_turn>model\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, t

现在让我们打印经过掩码处理的示例 - 您应该只能看到答案部分：

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                               In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:\n\n```\nModulus of the given numbers is: 2\n```\n\nThis means that the modulus of 10 and 4 is 2.<end_of_t

In [48]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.57 GB of memory reserved.


让我们开始训练模型！如需从检查点恢复训练，请设置 `trainer.train(resume_from_checkpoint = True)`

In [50]:
trainer_stats = trainer.train(resume_from_checkpoint = False)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,185 | Num Epochs = 8 | Total steps = 3,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 6
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 6 x 1) = 24
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Step,Training Loss
1,1.474300
2,1.461900
3,1.608200
4,1.475800
5,1.485600
6,1.814800
7,1.644600


KeyboardInterrupt: 

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1068.4322 seconds used for training.
17.81 minutes used for training.
Peak reserved memory = 13.561 GB.
Peak reserved memory for training = 9.278 GB.
Peak reserved memory % of max memory = 91.995 %.
Peak reserved memory for training % of max memory = 62.94 %.


<a name="Inference"></a>
### 模型推理

现在让我们通过Unsloth原生推理接口运行模型！根据`Gemma-3`团队的推荐，最佳推理参数设置为：`temperature = 1.0`、`top_p = 0.95`、`top_k = 64`。

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89...\n\nThis is the Fibonacci sequence, where each number is the sum of the two preceding ones.\n<end_of_turn>']

您还可以使用 `TextStreamer` 进行连续推理 —— 这样您就可以实时看到每个token的生成过程，而无需等待完整输出！

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down why the sky is blue! It's a fascinating phenomenon that boils down to a combination of physics and light. Here's the explanation:

**1. Sunlight and its Colors:**

* Sunlight, which appears white to us, is actually made up of *all* the


<a name="Save"></a>
### 保存与加载微调模型

如需将最终模型保存为LoRA适配器，可选择：
- 使用Huggingface的`push_to_hub`进行在线保存
- 或使用`save_pretrained`进行本地保存

**[注意]** 此操作仅保存LoRA适配器，而非完整模型。如需保存为16bit或GGUF格式，请查看下方内容！

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3/processor_config.json']

现在，如果您想加载我们刚保存的LoRA适配器进行推理，请将`False`设为`True`：

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### 保存为 float16 格式（适用于 VLLM）

我们同样支持直接保存为 `float16` 格式以便部署！模型将保存至 `gemma-3-finetune` 文件夹。请将条件语句 `if False` 改为 `if True` 以执行该操作！

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

如需上传至您的 Hugging Face 账户，请将 `if False` 改为 `if True`，并添加您的 Hugging Face 访问令牌及上传路径！

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF/llama.cpp 格式转换

我们现已原生支持所有模型保存为`GGUF`/`llama.cpp`格式！当前可轻松转换为`Q8_0`、`F16`或`BF16`精度，`Q4_K_M`(4比特量化)支持即将推出！

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

同样，如果您想将 GGUF 推送到您的 Hugging Face 账户，请将  `if False`设置为`if True` 并添加您的 Hugging Face 标记和上传位置！

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

现在，在 llama.cpp 或基于 UI 的系统（如 Jan 或 Open WebUI）中使用 `gemma-3-finetune.gguf` 文件或 `gemma-3-finetune-Q4_K_M.gguf` 文件。您可以安装 Jan [此处](https://github.com/janhq/jan) 和 Open WebUI [此处](https://github.com/open-webui/open-webui)

这样就完成了！如果你有任何关于 Unsloth 的问题，我们有一个 [Discord](https://discord.gg/unsloth) 频道！如果你发现任何 Bug，或者想了解 LLM 的最新动态，或者需要帮助、加入项目等，请随时加入我们的 Discord！

其他链接
1. 训练自己的推理模型 - Llama GRPO 笔记本 [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. 将微调保存到 Ollama。[Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography 用例。[Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. 请参阅我们[文档](https://docs.unsloth.ai/get-started/unsloth-notebooks)上的 DPO、ORPO、持续预训练、会话微调等笔记本！现在，在 llama.cpp 或基于 UI 的系统（如 Jan 或 Open WebUI）中使用 `gemma-3-finetune.gguf` 文件或 `gemma-3-finetune-Q4_K_M.gguf` 文件。您可以安装 Jan [此处](https://github.com/janhq/jan) 和 Open WebUI [此处](https://github.com/open-webui/open-webui)

这样就完成了！如果你有任何关于 Unsloth 的问题，我们有一个 [Discord](https://discord.gg/unsloth) 频道！如果你发现任何 Bug，或者想了解 LLM 的最新动态，或者需要帮助、加入项目等，请随时加入我们的 Discord！

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  如果需要帮助，请加入 Discord + ⭐️ <i>在<a href="https://github.com/unslothai/unsloth">Github</a>上与我们联系</i> ⭐️
</div>